In [77]:
import torch
import time

# Set random seed for reproducibility
torch.manual_seed(5)

# Create tensors
a = torch.randint(low=0, high=100, size=(1000, 1000))  # 1000x1000 tensor of integers [0, 100)
b = torch.randint(low=0, high=100, size=(1000, 1000))  # 1000x1000 tensor of integers [0, 100)

# CPU matrix multiplication
start_time = time.time()
multiply = torch.matmul(a, b)
print("CPU time:", time.time() - start_time, "seconds")

# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device('cuda')
    # Cast tensors to float32 before moving to GPU
    a = a.to(device, dtype=torch.float32)
    b = b.to(device, dtype=torch.float32)
    
    # GPU matrix multiplication
    start_time = time.time()
    multiply = torch.matmul(a, b)
    torch.cuda.synchronize()  # Ensure GPU operations are complete
    print("GPU time:", time.time() - start_time, "seconds")
else:
    print("CUDA is not available. Skipping GPU computation.")

CPU time: 9.163564205169678 seconds
GPU time: 0.007004261016845703 seconds


In [91]:
#autograd
x=torch.tensor(3.0,requires_grad=True)
y=x**2
z=torch.sin(y)
print(x,y,z)
z.backward()
print(x.grad)

tensor(3., requires_grad=True) tensor(9., grad_fn=<PowBackward0>) tensor(0.4121, grad_fn=<SinBackward0>)
tensor(-5.4668)


In [123]:
def cross_entropy_loss(y_pred,y):
    epsilon=1e-8
    prediction=torch.clamp(y_pred,epsilon,1-epsilon)
    return -(y*torch.log(prediction)+(1-y)*torch.log(1-prediction))
x=torch.tensor(6.79)
y=torch.tensor(0.0)
w=torch.tensor(1.0,requires_grad=True)
b=torch.tensor(0.0,requires_grad=True)
z=w*x+b
y_pred=torch.sigmoid(z)
loss=cross_entropy_loss(y_pred,y)
loss.backward()
print(w.grad)
print(b.grad)
w.grad.zero_()
b.grad.zero_()

tensor(6.7824)
tensor(0.9989)


tensor(0.)

In [ ]:
#to stop the gradient calculations we can use
#requires_grad_(False)

In [101]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.drop(columns=["id","Unnamed: 32"],inplace=True)
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
encoder=LabelEncoder()
y_train=encoder.fit_transform(y_train)
y_test=encoder.transform(y_test)
X_train_tensor=torch.from_numpy(X_train).float()
X_test_tensor=torch.from_numpy(X_test).float()
y_train_tensor=torch.from_numpy(y_train).float()
y_test_tensor=torch.from_numpy(y_test).float()
learning_rate=0.1
epoches=25
weights=torch.rand(df.shape[1]-1,1,requires_grad=True)
bias=torch.rand(1,requires_grad=True)

def cross_entropy_loss(y_pred,y):
    epsilon=1e-8
    prediction=torch.clamp(y_pred,epsilon,1-epsilon)
    return -(y*torch.log(prediction)+(1-y)*torch.log(1-prediction))
for epoch in range(epoches):
    loss=0
    for i in range(X_train.shape[0]):
        z=torch.matmul(X_train_tensor[i],weights)+bias
        y_pred=torch.sigmoid(z)
        loss+=cross_entropy_loss(y_pred,y_train_tensor[i])
    print(loss)
    loss.backward()
    with torch.no_grad():
        weights -= learning_rate * weights.grad
        bias -= learning_rate * bias.grad
    weights.grad.zero_()
    bias.grad.zero_()

tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)


In [8]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load and preprocess the data
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.drop(columns=["id", "Unnamed: 32"], inplace=True)

X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

X_train_tensor = torch.from_numpy(X_train).float()
X_test_tensor = torch.from_numpy(X_test).float()
y_train_tensor = torch.from_numpy(y_train).float().unsqueeze(1)  # Make it (n_samples, 1)
y_test_tensor = torch.from_numpy(y_test).float().unsqueeze(1)

# Initialize weights and bias
learning_rate = 0.1
epochs = 25
weights = torch.randn(X_train.shape[1], 1, requires_grad=True)
bias = torch.randn(1, requires_grad=True)

# Define loss
def cross_entropy_loss(y_, y):
    epsilon = 1e-8
    y_pred = torch.clamp(y_, epsilon, 1 - epsilon)
    return -(y * torch.log(y_pred) + (1 - y) * torch.log(1 - y_pred))

# Training loop
for epoch in range(epochs):
    total_loss = 0
    for i in range(X_train_tensor.shape[0]):
        z = torch.matmul(X_train_tensor[i], weights) + bias
        y_pred = torch.sigmoid(z)
        loss = cross_entropy_loss(y_pred, y_train_tensor[i])
        total_loss += loss
    print(total_loss)
    total_loss.backward()  # Backward pass on total (scalar) loss

    # Gradient descent update
    with torch.no_grad():
        weights -= learning_rate * weights.grad
        bias -= learning_rate * bias.grad

        # Zero gradients
        weights.grad.zero_()
        bias.grad.zero_()


tensor([400.3140], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)
tensor([nan], grad_fn=<AddBackward0>)


In [25]:
import torch
import torch.nn as nn
class Model(nn.Module):
    def __init__(self,num_features):
        super().__init__()
        self.linear=nn.Linear(num_features,1)
        self.sigmoid=nn.Sigmoid()
    def forward(self,features):
        out=self.linear(features)
        out=self.sigmoid(out)

        return out

features=torch.rand(10,5)
model=Model(features.shape[1])
model(features)

tensor([[0.4627],
        [0.4433],
        [0.4141],
        [0.4099],
        [0.3908],
        [0.4654],
        [0.4179],
        [0.5111],
        [0.4817],
        [0.4740]], grad_fn=<SigmoidBackward0>)

In [63]:
#dataset and dataloader classes
from sklearn.datasets import make_classification
import torch
from torch.utils.data import Dataset,DataLoader
X,y=make_classification(
    n_samples=10,
    n_features=2,
    n_informative=2,
    n_redundant=0,
    n_classes=2,
    random_state=10)
X=torch.tensor(X,dtype=torch.float32)
y=torch.tensor(y,dtype=torch.long)

class CustomDataset(Dataset):
    def __init__(self,features,labels):
        self.features=features
        self.labels=labels
    def __len__(self):
        return self.features.shape[0]
    def __getitem__(self,index):
        return self.features[index],self.labels[index]
dataset=CustomDataset(X,y)
len(dataset)
dataset[0]
dataloader=DataLoader(dataset,batch_size=2,shuffle=True)
for batch_features,batch_labels in dataloader:
    print(batch_features)
    print(batch_labels)

tensor([[ 1.7031,  0.8324],
        [ 0.9933, -1.1053]])
tensor([1, 1])
tensor([[-1.2601,  0.8223],
        [-0.7963, -0.8222]])
tensor([0, 0])
tensor([[-1.1009, -1.2419],
        [ 1.0584, -0.8539]])
tensor([0, 1])
tensor([[ 1.1396, -0.1354],
        [-1.1569, -0.9864]])
tensor([1, 0])
tensor([[-1.0610, -0.6310],
        [ 0.2870,  1.7550]])
tensor([0, 1])
